In [1]:
import pyspark
import pandas as pd
import numpy as np
from numpy import *
from numpy import inf


from pyspark.sql import SparkSession, HiveContext
spark = SparkSession.builder.appName('Ops').getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType

from datetime import date
import datetime
import glob
import os
from datetime import datetime
import pytz


In [2]:
%run /Shared/psbu_qa_mining/common/config_ADLS

In [3]:
%scala
//Get the run id in Scala and pass it in via the temp table:
val runId = dbutils.notebook.getContext .currentRunId .getOrElse(System.currentTimeMillis() / 1000L) .toString
Seq(runId).toDF("run_id").createOrReplaceTempView("run_id")

runId: String = 1563283411

In [4]:
myclusterId= spark.conf.get("spark.databricks.clusterUsageTags.clusterId")
myclusterName= spark.conf.get("spark.databricks.clusterUsageTags.clusterName")
myrunId = spark.table("run_id").head()["run_id"]

In [5]:
TestFlag=[True] * 25
CriticalTestFlag=[True] * 25

In [6]:
CriticalTable = {'DS_POS_OUT_001': 'Not_Critical', 'DS_POS_OUT_002': 'Not_Critical', 'DS_POS_OUT_003': 'Not_Critical', 'DS_POS_OUT_004': 'Not_Critical', 'DS_POS_OUT_005': 'Not_Critical', 'DS_POS_OUT_006': 'Not_Critical','DS_POS_OUT_007': 'Not_Critical', 'DS_POS_OUT_008': 'Not_Critical', 'DS_POS_OUT_009': 'Not_Critical', 'DS_POS_OUT_010': 'Not_Critical', 'DS_POS_OUT_011': 'Not_Critical', 'DS_POS_OUT_012': 'Not_Critical', 'DS_POS_OUT_013': 'Not_Critical', 'DS_POS_OUT_014': 'Not_Critical', 'DS_POS_OUT_015': 'Not_Critical', 'DS_POS_OUT_016': 'Not_Critical', 'DS_POS_OUT_017': 'Not_Critical', 'DS_POS_OUT_018': 'Not_Critical', 'DS_POS_OUT_019': 'Not_Critical', 'DS_POS_OUT_020': 'Not_Critical', 'DS_POS_OUT_021': 'Not_Critical', 'DS_POS_OUT_022': 'Not_Critical', 'DS_POS_OUT_023': 'Not_Critical', 'DS_POS_OUT_024': 'Not_Critical', 'DS_POS_OUT_025': 'Not_Critical'}


In [7]:
%fs ls adl://eaasedldevadls.azuredatalakestore.net/data/mo/quality_engine/om_output/

path,name,size
adl://eaasedldevadls.azuredatalakestore.net/data/mo/quality_engine/om_output/OM_MODEL_OUTPUT_2019-02-25.csv,OM_MODEL_OUTPUT_2019-02-25.csv,701397002
adl://eaasedldevadls.azuredatalakestore.net/data/mo/quality_engine/om_output/OM_MODEL_OUTPUT_2019-02-26.csv,OM_MODEL_OUTPUT_2019-02-26.csv,701397002
adl://eaasedldevadls.azuredatalakestore.net/data/mo/quality_engine/om_output/OM_MODEL_OUTPUT_2019-03-05.csv,OM_MODEL_OUTPUT_2019-03-05.csv,701654799
adl://eaasedldevadls.azuredatalakestore.net/data/mo/quality_engine/om_output/OM_MODEL_OUTPUT_2019-03-06.csv,OM_MODEL_OUTPUT_2019-03-06.csv,2322507326
adl://eaasedldevadls.azuredatalakestore.net/data/mo/quality_engine/om_output/OM_MODEL_OUTPUT_2019-03-07.csv,OM_MODEL_OUTPUT_2019-03-07.csv,2248394825
adl://eaasedldevadls.azuredatalakestore.net/data/mo/quality_engine/om_output/OM_MODEL_OUTPUT_2019-03-12.csv,OM_MODEL_OUTPUT_2019-03-12.csv,2248771093
adl://eaasedldevadls.azuredatalakestore.net/data/mo/quality_engine/om_output/OM_MODEL_OUTPUT_2019-03-20.csv,OM_MODEL_OUTPUT_2019-03-20.csv,2667257436
adl://eaasedldevadls.azuredatalakestore.net/data/mo/quality_engine/om_output/OM_MODEL_OUTPUT_2019-04-01.csv,OM_MODEL_OUTPUT_2019-04-01.csv,433834574
adl://eaasedldevadls.azuredatalakestore.net/data/mo/quality_engine/om_output/OM_MODEL_OUTPUT_2019-04-11.csv,OM_MODEL_OUTPUT_2019-04-11.csv,295922235
adl://eaasedldevadls.azuredatalakestore.net/data/mo/quality_engine/om_output/OM_MODEL_OUTPUT_2019-04-18.csv,OM_MODEL_OUTPUT_2019-04-18.csv,405212235


In [8]:
POS_OUT_XBL = spark.read.option("header","true").option("delimiter","|").option("inferSchema","true").format("csv").load("adl://eaasedldevadls.azuredatalakestore.net/data/mo/quality_engine/om_output/OM_MODEL_OUTPUT_2019-06-27.csv")
# Creating Pandas df from Spark df
POS_OUT_XBL_df = POS_OUT_XBL.toPandas()

display(POS_OUT_XBL)

CALC_ID,CALC_DATE,CODE,OM_CRRT_PRIORITY_SCORE,OM_PREV_PRIORITY_SCORE,OM_OUTPUT_01,OM_OUTPUT_02,OM_OUTPUT_03,SOURCE,OM_ALGO,OM_RUN_ANALYTICS_DATE,OM_NBR,OM_NBR_PREV,OM_SCORE,OM_SCORE_DELTA,OM_SCORE_REL,OM_SCORE_REL_PREV,OM_SCORE_REL_BUCKET,OM_SCORE_REL_PREV_BUCKET,OM_WEIBULL_SCORE_RPH_MILEAGE_MEDIAN,OM_WEIBULL_SCORE_RPH_MONTH_IN_SERVICE_MEDIAN,ENGINE_SERIES,REL_ENGINE_NAME_DESC,PROGRAM_GROUP_NAME,OM_DELTA,OM_METRIC,OM_KEY,OM_JIRA_URL,BUILD_VOLUME,BUILD_YEAR,REL_USER_APPL_DESC,EARLIEST_OCCURRENCE_DATE,OM_NBR_CUMUL,OM_NBR_CUMUL_BUCKET,OM_CRRT_PRIORITY_SCORE_BUCKET,REL_MONTH_BUILD_DATE,REL_OEM_NORMALIZED_GROUP,WEIBULL_RPH_MONTH_IN_SERVICE_MEDIAN,WEIBULL_RPH_MILEAGE_MEDIAN,REL_QUARTER_BUILD_DATE
111-All-X-All-All-All-2015-Q1-2015-TELEMATICS,2017-07-06T00:00:00.000+0000,111,12.8159884164006,0.0,null,null,null,TELEMATICS,MA_Ribbon,2019-06-30T00:00:00.000+0000,8.0,0.0,2.1359980694001104,4.3346898338653705,2.1359980694001104,2.12032464563382,High,High,null,null,X,All,All,12.8159884164006,COUNT_NBR,111-All-X-All-All-All-2015-Q1-2015-TELEMATICS-X-All-COUNT_NBR-MA_Ribbon,https://cumminsit.atlassian.net/secure/CreateIssueDetails!init.jspa?pid=10047&issuetype=10023&summary=0111:%20TELEMATICS-COUNT_NBR&description=OCCURRENCE%20DATA%0AName%20of%20the%20Metric:%208.000%0ADelta:%2012.8159884164006%0A&customfield_10093=0111&customfield_10030=10197,1111,2015,All,null,16.0,Middle,High,All,All,null,null,2015-Q1
111-All-X-All-All-All-2016-Q3-2016-TELEMATICS,2017-07-06T00:00:00.000+0000,111,12.244444444445099,0.0,null,null,null,TELEMATICS,MA_Ribbon,2019-06-30T00:00:00.000+0000,8.0,2.0,2.04074074074086,4.34444444444305,3.06111111111129,3.9500000000010496,High,High,null,null,X,All,All,12.244444444445099,COUNT_NBR,111-All-X-All-All-All-2016-Q3-2016-TELEMATICS-X-All-COUNT_NBR-MA_Ribbon,https://cumminsit.atlassian.net/secure/CreateIssueDetails!init.jspa?pid=10047&issuetype=10023&summary=0111:%20TELEMATICS-COUNT_NBR&description=OCCURRENCE%20DATA%0AName%20of%20the%20Metric:%208.000%0ADelta:%2012.244444444445099%0A&customfield_10093=0111&customfield_10030=10197,6840,2016,All,null,16.0,Middle,High,All,All,null,null,2016-Q3
111-All-X-All-All-All-All-2013-TELEMATICS,2017-07-06T00:00:00.000+0000,111,134.750308641975,0.0,null,null,null,TELEMATICS,MA_Ribbon,2019-06-30T00:00:00.000+0000,54.0,0.0,22.4583847736626,3.87438271604941,1.42592919197857,1.61575217192501,Middle,Middle,null,null,X,All,All,134.750308641975,COUNT_NBR,111-All-X-All-All-All-All-2013-TELEMATICS-X-All-COUNT_NBR-MA_Ribbon,https://cumminsit.atlassian.net/secure/CreateIssueDetails!init.jspa?pid=10047&issuetype=10023&summary=0111:%20TELEMATICS-COUNT_NBR&description=OCCURRENCE%20DATA%0AName%20of%20the%20Metric:%2054.000%0ADelta:%20134.750308641975%0A&customfield_10093=0111&customfield_10030=10197,3068,2013,All,null,378.0,High,High,All,All,null,null,All
111-All-X-All-All-All-All-2014-TELEMATICS,2017-07-06T00:00:00.000+0000,111,122.633333333333,0.0,null,null,null,TELEMATICS,MA_Ribbon,2019-06-30T00:00:00.000+0000,54.0,0.0,20.438888888888897,18.6583333333334,2.27098765432099,2.5672839506172798,High,High,null,null,X,All,All,122.633333333333,COUNT_NBR,111-All-X-All-All-All-All-2014-TELEMATICS-X-All-COUNT_NBR-MA_Ribbon,https://cumminsit.atlassian.net/secure/CreateIssueDetails!init.jspa?pid=10047&issuetype=10023&summary=0111:%20TELEMATICS-COUNT_NBR&description=OCCURRENCE%20DATA%0AName%20of%20the%20Metric:%2054.000%0ADelta:%20122.633333333333%0A&customfield_10093=0111&customfield_10030=10197,4161,2014,All,null,216.0,High,High,All,All,null,null,All
111-All-X-All-All-All-All-2015-TELEMATICS,2017-07-06T00:00:00.000+0000,111,138.59176954732501,0.0,null,null,null,TELEMATICS,MA_Ribbon,2019-06-30T00:00:00.000+0000,54.0,0.0,23.0986282578875,3.8414609053497797,1.2832571254382,1.42592919197857,Middle,Middle,null,null,X,All,All,138.59176954732501,COUNT_NBR,111-All-X-All-All-All-All-2015-TELEMATICS-X-All-COUNT_NBR-MA_Ribbon,https://cumminsit.atlassian.net/secure/CreateIssueDetails!init.jspa?pid=10047&issuetype=10023&summar

In [9]:
POS_OUT_XBL.printSchema()

root
-- CALC_ID: string (nullable = true)
-- CALC_DATE: timestamp (nullable = true)
-- CODE: string (nullable = true)
-- OM_CRRT_PRIORITY_SCORE: double (nullable = true)
-- OM_PREV_PRIORITY_SCORE: double (nullable = true)
-- OM_OUTPUT_01: timestamp (nullable = true)
-- OM_OUTPUT_02: double (nullable = true)
-- OM_OUTPUT_03: double (nullable = true)
-- SOURCE: string (nullable = true)
-- OM_ALGO: string (nullable = true)
-- OM_RUN_ANALYTICS_DATE: timestamp (nullable = true)
-- OM_NBR: double (nullable = true)
-- OM_NBR_PREV: double (nullable = true)
-- OM_SCORE: double (nullable = true)
-- OM_SCORE_DELTA: double (nullable = true)
-- OM_SCORE_REL: double (nullable = true)
-- OM_SCORE_REL_PREV: double (nullable = true)
-- OM_SCORE_REL_BUCKET: string (nullable = true)
-- OM_SCORE_REL_PREV_BUCKET: string (nullable = true)
-- OM_WEIBULL_SCORE_RPH_MILEAGE_MEDIAN: double (nullable = true)
-- OM_WEIBULL_SCORE_RPH_MONTH_IN_SERVICE_MEDIAN: double (nullable = true)
-- ENGINE_SERIES: string (nullable = true)
-- REL_ENGINE_NAME_DESC: string (nullable = true)
-- PROGRAM_GROUP_NAME: string (nullable = true)
-- OM_DELTA: double (nullable = true)
-- OM_METRIC: string (nullable = true)
-- OM_KEY: string (nullable = true)
-- OM_JIRA_URL: string (nullable = true)
-- BUILD_VOLUME: integer (nullable = true)
-- BUILD_YEAR: string (nullable = true)
-- REL_USER_APPL_DESC: string (nullable = true)
-- EARLIEST_OCCURRENCE_DATE: timestamp (nullable = true)
-- OM_NBR_CUMUL: double (nullable = true)
-- OM_NBR_CUMUL_BUCKET: string (nullable = true)
-- OM_CRRT_PRIORITY_SCORE_BUCKET: string (nullable = true)
-- REL_MONTH_BUILD_DATE: string (nullable = true)
-- REL_OEM_NORMALIZED_GROUP: string (nullable = true)
-- WEIBULL_RPH_MONTH_IN_SERVICE_MEDIAN: double (nullable = true)
-- WEIBULL_RPH_MILEAGE_MEDIAN: double (nullable = true)
-- REL_QUARTER_BUILD_DATE: string (nullable = true)

In [10]:
# ************************** DS_POS_OUT_001
# DS_POS_OUT_001 :: Scenario :: The Column 'REL_CMP_ENGINE_NAME' should have 'X', 'B' or 'L' values.

def DS_POS_OUT_001(POS_OUT_XBL_df):
  ENgine_Name = POS_OUT_XBL_df['ENGINE_SERIES'].unique().tolist()
  c1=0 
  ls1=[]
  for i in ENgine_Name:    
    if i in ['X', 'B','L'] and len(ENgine_Name) == 3: 
      c1+=1
    else:
      ls1.append(i)
  #print(ls1) 
  if c1 == 3: 
      print("DS_POS_OUT_001 :: For POS_OUT_XBL_df test case passed")
      log ='Test case passed'
  else:
      print("DS_POS_OUT_001 :: For POS_OUT_XBL_df test case failed")
      log ='Column ENGINE_SERIES is having other values also :: Test case Failed' 
      TestFlag[1]=False
      CriticalTestFlag[1]=TestFlag[1] and CriticalTable['DS_POS_OUT_001']=="Critical"      
  return TestFlag[1],log

my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_001(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_001'
nature_of_tc=CriticalTable['DS_POS_OUT_001']
metrics= "ENGINE_SERIES"
value='ENGINE_SERIES column should have only X, B or L as values'
if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log =str(sree_o[1])

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")
  

DS_POS_OUT_001 :: For POS_OUT_XBL_df test case passed
 Out[ 10 ]: DataFrame[]

In [11]:
# *********************************** DS_POS_OUT_002
# Scenario :: For POS_OUT_XBL_df, The 'CALC_ID' column should be in the format - CODE-REL_USER_APPL_DESC-ENGINE_SERIES-REL_ENGINE_NAME_DESC-REL_OEM_NORMALIZED_GROUP-REL_MONTH_BUILD_DATE-REL_QUARTER_BUILD_DATE-BUILD_YEAR-SOURCE

def DS_POS_OUT_002(POS_OUT_XBL_df):
    calc_id = POS_OUT_XBL_df['CALC_ID'].tolist()
    New_CALC_ID=POS_OUT_XBL_df['CODE'].astype(str)+'-'+POS_OUT_XBL_df['REL_USER_APPL_DESC'].astype(str)+'-'+POS_OUT_XBL_df['ENGINE_SERIES'].astype(str)+'-'+POS_OUT_XBL_df['REL_ENGINE_NAME_DESC'].astype(str)+'-'+POS_OUT_XBL_df['REL_OEM_NORMALIZED_GROUP'].astype(str)+'-'+POS_OUT_XBL_df['REL_MONTH_BUILD_DATE'].astype(str)+'-'+POS_OUT_XBL_df['REL_QUARTER_BUILD_DATE'].astype(str)+'-'+POS_OUT_XBL_df['BUILD_YEAR'].astype(str)+'-'+POS_OUT_XBL_df['SOURCE'].astype(str)
    if len(calc_id) == len(New_CALC_ID):
        c=0
        ls=[]
        for i in range(0,len(New_CALC_ID)):
            if calc_id[i] == New_CALC_ID[i]:
                c+=1
            else:
                ls.append(i)
        if c == len(calc_id) and len(ls)==0 :
            print("DS_POS_OUT_002 :: For POS_OUT_XBL_df Test case pass")
            log ='Test Case Passes'
        else:
            print("DS_POS_OUT_002 :: For POS_OUT_XBL_df Test case fail")
            log ='The Column "CALC_ID" not in specifed combination ::  Test Case Fail'
            TestFlag[2]=False
            CriticalTestFlag[2]=TestFlag[2] and CriticalTable['DS_POS_OUT_002']=="Critical"
        return TestFlag[2],log
    
my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_002(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_002'
nature_of_tc=CriticalTable['DS_POS_OUT_002']
metrics= "CALC_ID"
value='CALC_ID should be in specified format'
if sree_o[0] :
  status = 'Pass'
else:
  status = 'Fail'
failure_log = str(sree_o[1])

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")                


DS_POS_OUT_002 :: For POS_OUT_XBL_df Test case fail
 Out[ 14 ]: DataFrame[]

In [12]:
# *********************************** DS_POS_OUT_003
# Scenario :: CALC_DATE should be the most recent Thursday's date for all engine series.
def DS_POS_OUT_003(POS_OUT_XBL_df):    
    Max_Calc_Date = POS_OUT_XBL_df.CALC_DATE.max()       
    #print(engine)
    #st = df.select('').flatMap(lambda x: x).collect()
    print("DS_POS_OUT_003 :: For POS_OUT_XBL_df Max calc_date is " + str(Max_Calc_Date))
    return Max_Calc_Date
    
my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o=  DS_POS_OUT_003(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_003'
nature_of_tc="Highly_Critical"
metrics= "CALC_DATE"
value='CALC_DATE should be Recent in data'
test_status='Pass'
failure_log = ' For POS_OUT_XBL_df  Max_calc_date  in Output data is == ' + str(sree_o)

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")   

DS_POS_OUT_003 :: For POS_OUT_XBL_df Max calc_date is 2019-06-27 00:00:00
 Out[ 15 ]: DataFrame[]

In [13]:
# ********************************** DS_POS_OUT_004
# Scenario :: The OM_CRRT_PRIORITY_SCORE, OM_NBR, OM_NBR_PREV, OM_SCORE, OM_SCORE_REL, OM_DELTA, OM_NBR_CUMUL, BUILD_VOLUME, WEIBULL_RPH_MONTH_IN_SERVICE_MEDIAN, WEIBULL_RPH_MILEAGE_MEDIAN columns should have positive and non-infinite values.

def DS_POS_OUT_004(POS_OUT_XBL_df):
  import numpy as np
  from numpy import inf
  col_lst = ['OM_CRRT_PRIORITY_SCORE', 'OM_NBR, OM_NBR_PREV', 'OM_SCORE', 'OM_SCORE_REL', 'OM_DELTA', 'OM_NBR_CUMUL', 'BUILD_VOLUME', 'WEIBULL_RPH_MONTH_IN_SERVICE_MEDIAN', 'WEIBULL_RPH_MILEAGE_MEDIAN']
  col_names = POS_OUT_XBL_df.columns.tolist()
  col_names = [str.upper(cval) for cval in col_names]
  count_lst=[]
  for i in col_names:
    if i in col_lst :
      count_lst.append(i)   
  def infinite_check(data_1):
    # If infinite values present it is true otherwise false
    if np.isinf(POS_OUT_XBL_df[data_1].astype(float)).any() and Pre_X_W_df.filter(Pre_X_W_df[data_f].astype(float)<0).count() == 0 :
      sree = 1
      return sree
  c=0
  c1=0
  ls1=[]
  ls2=[]        
  for i in count_lst:
    if infinite_check(i) != 1:
      c+=1
      ls1.append(i)
    else:
      c1+=1
      ls2.append(i)        
  print(c,c1)
  r_ls=[]
  if c == len(count_lst) and c1 == 0:
    print('All Columns are passes')
    log="For All Columns Test case passed" 
    r_ls.append(log)
    for i in ls1 :
      print("DS_POS_OUT_004 :: For POS_OUT_XBL_df Column name == " + i + " == Test case passed")      
  else:
    print('Some columns are passed some are failed')
    TestFlag[4]=False
    CriticalTestFlag[4]=TestFlag[4] and CriticalTable['DS_POS_OUT_004']=="Critical"
    for i in ls1 :
      print(" DS_POS_OUT_004 :: Column == " + i + " == Test case passed")
      log=" For Column == " + i + " == Test case passed"
      r_ls.append(log)
    for j in ls2 :
      print(" DS_POS_OUT_004 :: Column == " + j + " == Test case Failed")
      log=" For Column  == " + j + " == Test case Failed"  
      r_ls.append(log)
  print(len(ls1),len(ls2))
  return TestFlag[4],r_ls

my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_004(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_004'
nature_of_tc=CriticalTable["DS_POS_OUT_004"]
metrics= "OM_CRRT_PRIORITY_SCORE, OM_NBR, OM_NBR_PREV, OM_SCORE, OM_SCORE_REL, OM_DELTA, OM_NBR_CUMUL, BUILD_VOLUME, WEIBULL_RPH_MONTH_IN_SERVICE_MEDIAN, WEIBULL_RPH_MILEAGE_MEDIAN columns"
value='Columns values should have positive and non infinite values'
if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log=str(sree_o[1])
df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df") 

8 0
All Columns are passes
DS_POS_OUT_004 :: For POS_OUT_XBL_df Column name == OM_CRRT_PRIORITY_SCORE == Test case passed
DS_POS_OUT_004 :: For POS_OUT_XBL_df Column name == OM_SCORE == Test case passed
DS_POS_OUT_004 :: For POS_OUT_XBL_df Column name == OM_SCORE_REL == Test case passed
DS_POS_OUT_004 :: For POS_OUT_XBL_df Column name == OM_DELTA == Test case passed
DS_POS_OUT_004 :: For POS_OUT_XBL_df Column name == BUILD_VOLUME == Test case passed
DS_POS_OUT_004 :: For POS_OUT_XBL_df Column name == OM_NBR_CUMUL == Test case passed
DS_POS_OUT_004 :: For POS_OUT_XBL_df Column name == WEIBULL_RPH_MONTH_IN_SERVICE_MEDIAN == Test case passed
DS_POS_OUT_004 :: For POS_OUT_XBL_df Column name == WEIBULL_RPH_MILEAGE_MEDIAN == Test case passed
8 0
 Out[ 16 ]: DataFrame[]

In [14]:
# ********************************** DS_POS_OUT_005
# Scenario :: The OM_PREV_PRIORITY_SCORE, OM_SCORE_DELTA, OM_SCORE_REL_PREV, OM_OUTPUT_2, OM_OUTPUT_3, OM_WEIBULL_SCORE_RPH_MILEAGE_MEDIAN, OM_WEIBULL_SCORE_RPH_MONTH_IN_SERVICE_MEDIAN columns should not have infinite values.

def DS_POS_OUT_005(POS_OUT_XBL_df):
  import numpy as np
  from numpy import inf
  count_lst = ['OM_PREV_PRIORITY_SCORE', 'OM_SCORE_DELTA', 'OM_SCORE_REL_PREV', 'OM_OUTPUT_02', 'OM_OUTPUT_03', 'OM_WEIBULL_SCORE_RPH_MILEAGE_MEDIAN', 'OM_WEIBULL_SCORE_RPH_MONTH_IN_SERVICE_MEDIAN']  
  def infinite_check(data_1):
    # If infinite values present it is true otherwise false
    if np.isinf(POS_OUT_XBL_df[data_1].astype(float)).any() :
      sree = 1
      return sree
  c=0
  c1=0
  ls1=[]
  ls2=[]        
  for i in count_lst:
    if infinite_check(i) != 1:
      c+=1
      ls1.append(i)
    else:
      c1+=1
      ls2.append(i)        
  print(c,c1)
  r_ls=[]
  if c == len(count_lst) and c1 == 0:
    print('All COlumns are passes')
    log="For All Columns Test case passed" 
    r_ls.append(log)
    for i in ls1 :
      print("DS_POS_OUT_005  :: For POS_OUT_XBL_df Column name == " + i + " == Test case passed")        
  else:
    print('Some columns are passed some are failed')
    TestFlag[5]=False
    CriticalTestFlag[5]=TestFlag[5] and CriticalTable['DS_POS_OUT_005']=="Critical"
    for i in ls1 :
      print(" DS_POS_OUT_005 :: For POS_OUT_XBL_df Column name == " + i + " == Test case passed")
      log=" For Column == " + i + " == Test case passed"
      r_ls.append(log)
    for j in ls2 :
      print(" DS_POS_OUT_005 :: For POS_OUT_XBL_df Column name == " + j + " == Test case Failed")
      log=" For Column  == " + j + " == Test case Failed"  
      r_ls.append(log)
  print(len(ls1),len(ls2))
  return TestFlag[5],r_ls
my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_005(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_005'
nature_of_tc=CriticalTable["DS_POS_OUT_005"]
metrics= "OM_PREV_PRIORITY_SCORE, OM_SCORE_DELTA, OM_SCORE_REL_PREV, OM_OUTPUT_2, OM_OUTPUT_3, OM_WEIBULL_SCORE_RPH_MILEAGE_MEDIAN, OM_WEIBULL_SCORE_RPH_MONTH_IN_SERVICE_MEDIAN Columns"
value='Columns should have non infinite values'
if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log=str(sree_o[1])
df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df") 

7 0
All COlumns are passes
DS_POS_OUT_005 :: For POS_OUT_XBL_df Column name == OM_PREV_PRIORITY_SCORE == Test case passed
DS_POS_OUT_005 :: For POS_OUT_XBL_df Column name == OM_SCORE_DELTA == Test case passed
DS_POS_OUT_005 :: For POS_OUT_XBL_df Column name == OM_SCORE_REL_PREV == Test case passed
DS_POS_OUT_005 :: For POS_OUT_XBL_df Column name == OM_OUTPUT_02 == Test case passed
DS_POS_OUT_005 :: For POS_OUT_XBL_df Column name == OM_OUTPUT_03 == Test case passed
DS_POS_OUT_005 :: For POS_OUT_XBL_df Column name == OM_WEIBULL_SCORE_RPH_MILEAGE_MEDIAN == Test case passed
DS_POS_OUT_005 :: For POS_OUT_XBL_df Column name == OM_WEIBULL_SCORE_RPH_MONTH_IN_SERVICE_MEDIAN == Test case passed
7 0
 Out[ 17 ]: DataFrame[]

In [15]:
# ********************************** DS_POS_OUT_006
# Scenario :: if OM_ALGO is 'Trend_without_BB', then there should be data available for OM_OUTPUT_1/OM_OUTPUT_2/OM_OUTPUT_3

def DS_POS_OUT_006(POS_OUT_XBL_df):
  sk =POS_OUT_XBL_df[POS_OUT_XBL_df['OM_ALGO'].str.upper() == 'TREND_WITHOUT_BB']
  if len(sk)!=0:
    if pd.isnull(sk['OM_OUTPUT_01']).any() or pd.isnull(sk['OM_OUTPUT_02']).any() or pd.isnull(sk['OM_OUTPUT_03']).any() :
      print(' DS_POS_OUT_006 :: Test case Fail')
      log='For the Scenario Description :: Test case Fail'
      TestFlag[6]=False
      CriticalTestFlag[6]=TestFlag[6] and CriticalTable['DS_POS_OUT_006']=="Critical"      
    else:
      print(' DS_POS_OUT_006 ::Test case pass')
      log='Test case pass'      
    return TestFlag[6],log
  
my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_006(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_006'
nature_of_tc=CriticalTable["DS_POS_OUT_006"]
metrics= "OM_ALGO,OM_OUTPUT_1, OM_OUTPUT_3 and OM_OUTPUT_3 "
value='If OM_ALGO is Trend_without_BB, then there should be data available for OM_OUTPUT_1/OM_OUTPUT_2/OM_OUTPUT_3'
if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log =str(sree_o[1])
df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")   
    

DS_POS_OUT_006 ::Test case pass
 Out[ 18 ]: DataFrame[]

In [16]:
# ****************************** DS_POS_OUT_008
# Scenario 1 ::Column 'SOURCE' should have 'EDS', 'REL' or 'TELEMATICS'

def DS_POS_OUT_008(POS_OUT_XBL_df):
    dist_src= POS_OUT_XBL_df['SOURCE'].unique().tolist()
    src_1 =['REL','EDS','TELEMATICS']
    c=0
    ls=[]
    for i in dist_src:
        if i in src_1:
            c+=1
        else:
            ls.append(i)
    print(c,ls)    
    if len(ls) == 0 :
        print(' DS_POS_OUT_008 :: For POS_OUT_XBL_df Test case pass')
        log='Test case pass'
    else:        
        print(' DS_POS_OUT_008 :: For POS_OUT_XBL_df Test case fail')
        log=' SOURCE column is having values other than "EDS" or "REL" for X Output file :: Test case fail'
        TestFlag[8]=False
        CriticalTestFlag[8]=TestFlag[8] and CriticalTable['DS_POS_OUT_008']=="Critical"
    return TestFlag[8],log

my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_008(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_008'
nature_of_tc=CriticalTable["DS_POS_OUT_008"]
metrics= "SOURCE"
value='the SOURCE column should have only EDS or REL values'
if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log =str(sree_o[1])
df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")

3 []
 DS_POS_OUT_008 :: For POS_OUT_XBL_df Test case pass
 Out[ 19 ]: DataFrame[]

In [17]:
# ****************************** DS_POS_OUT_009
# Scenario :: The OM_ALGO column should have 'MA_Ribbon', Trend_with_BB or  'TREND_without_BB' as values.

def DS_POS_OUT_009(POS_OUT_XBL_df):
    dist_src= POS_OUT_XBL_df['OM_ALGO'].unique().tolist()
    src_1 =['MA_Ribbon','Trend_with_BB','TREND_without_BB']
    c=0
    ls=[]
    for i in dist_src:
        if i in src_1:
            c+=1
        else:
            ls.append(i)
    print(c,ls)    
    if len(ls) == 0 :
        print(' DS_POS_OUT_009 :: For POS_OUT_XBL_df Test case pass')
        log='Test case pass'
    else:        
        print(' DS_POS_OUT_009 :: For POS_OUT_XBL_df Test case fail')
        log=' OM_ALGO column is having values other values :: Test case fail'
        TestFlag[9]=False
        CriticalTestFlag[9]=TestFlag[9] and CriticalTable['DS_POS_OUT_009']=="Critical"
    return TestFlag[9],log

my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_009(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_009'
nature_of_tc=CriticalTable["DS_POS_OUT_009"]
metrics= "OM_ALGO"
value='the OM_ALGO column should have only MA_Ribbon, Trend_with_BB or TREND_without_BB as values'
if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log =str(sree_o[1])
df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")

2 []
 DS_POS_OUT_009 :: For POS_OUT_XBL_df Test case pass
 Out[ 20 ]: DataFrame[]

In [18]:
# *********************************** DS_POS_OUT_010
# Scenario :: OM_RUN_ANALYTICS_DATE column should have the most recent date on which the model has ran.
def DS_POS_OUT_010(POS_OUT_XBL_df): 
  #sk = POS_OUT_XBL_df['OM_RUN_ANALYTICS_DATE'].unique().tolist()  
    Max_Run_Date = POS_OUT_XBL_df.OM_RUN_ANALYTICS_DATE.max()       
    #print(engine)
    #st = df.select('').flatMap(lambda x: x).collect()
    print("DS_POS_OUT_010 :: For POS_OUT_XBL_df Max_Run_Date is " + str(Max_Run_Date))
    return Max_Run_Date
    
my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o=  DS_POS_OUT_010(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_010'
nature_of_tc="Highly_Critical"
metrics= "OM_RUN_ANALYTICS_DATE"
value='OM_RUN_ANALYTICS_DATE should be Recent in data'
test_status='Pass'
failure_log = ' For POS_OUT_XBL_df  Max_Run_Date  in Output data is == ' + str(sree_o)

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")   

DS_POS_OUT_010 :: For POS_OUT_XBL_df Max_Run_Date is 2019-06-30 00:00:00
 Out[ 21 ]: DataFrame[]

In [19]:
# *********************************** DS_POS_OUT_011
# The OM_SCORE_REL_BUCKET, OM_SCORE_REL_PREV_BUCKET, OM_NBR_CUMUL_BUCKET, OM_CRRT_PRIORITY_SCORE_BUCKET columns should have 'Low', 'Middle', 'High' or 'Decline'.. Columns should have all 3 or 4 values

def DS_POS_OUT_011(POS_OUT_XBL_df):  
  col_ls = ['OM_SCORE_REL_BUCKET', 'OM_SCORE_REL_PREV_BUCKET', 'OM_NBR_CUMUL_BUCKET', 'OM_CRRT_PRIORITY_SCORE_BUCKET']
  def val_check(col):
    v_ls = ['Low', 'Middle', 'High', 'Decline']
    Valus = POS_OUT_XBL_df[col].unique().tolist()
    for i in Valus:
      if i in v_ls and 3 <= len(Valus) <= 4:
        sree =1
        return sree
  s=0
  s1=0
  for i in col_ls:
    if val_check(i) == 1:
      s+=1
    else:
      s1+=1
  if s == len(col_ls):
    print("DS_POS_OUT_011 :: Test case pass")    
    log=" Test case pass"
  else:
    print("DS_POS_OUT_011 :: Test case fail")
    log="The Columns are not having specified values :: Test Case Fail"
    TestFlag[11]=False
    CriticalTestFlag[11]=TestFlag[11] and CriticalTable['DS_POS_OUT_011']=="Critical"
  return TestFlag[11],log

my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_011(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_011'
nature_of_tc=CriticalTable["DS_POS_OUT_011"]
metrics= "OM_SCORE_REL_BUCKET, OM_SCORE_REL_PREV_BUCKET, OM_NBR_CUMUL_BUCKET, OM_CRRT_PRIORITY_SCORE_BUCKET"
value='columns should have Low, Middle, High or Decline as values'
if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log=str(sree_o[1])
df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")

DS_POS_OUT_011 :: Test case pass
 Out[ 22 ]: DataFrame[]

In [20]:
# *********************************** DS_POS_OUT_012
# Scenario :: The Column REL_ENGINE_NAME_DESC sholud have  == 'All', 'ISX1 2013', 'ISX2 2013', 'ISX3 2013', 'X15 1 2017', 'X15 3 2017', 'B6.7 2017', 'ISB6.7 2013', 'ISL 2013', 'L9 2017' as values

def DS_POS_OUT_012(POS_OUT_XBL_df):
    # Listing the distinct values in output file
    list_given =['All', 'ISX1 2013', 'ISX2 2013', 'ISX3 2013', 'X15 1 2017', 'X15 3 2017', 'B6.7 2017', 'ISB6.7 2013', 'ISL 2013', 'L9 2017'] 
    dist_1 = POS_OUT_XBL_df['REL_ENGINE_NAME_DESC'].unique().tolist()
    s=0
    ls=[]
    for i in dist_1:
        if i in list_given:            
            s+=1
        else:
            ls.append(i)   
    if s == len(list_given) and len(ls) == 0:        
        print("DS_POS_OUT_012 ::FOR POS_OUT_XBL_df Test case pass")    
        log=" Test case pass"
    else:
        #log_file(ls)
        print("DS_POS_OUT_012 :: For POS_OUT_XBL_df Test case fail")
        log="The Column REL_ENGINE_NAME_DESC is having other values :: Test Case Fail"
        TestFlag[12]=False
        CriticalTestFlag[12]=TestFlag[12] and CriticalTable['DS_POS_OUT_012']=="Critical"
    return TestFlag[12],log
                        
my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_012(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_012'
nature_of_tc=CriticalTable["DS_POS_OUT_012"]
metrics= "REL_ENGINE_NAME_DESC"
value='REL_ENGINE_NAME_DESC colum should have only specified values'
if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log=str(sree_o[1])

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")

DS_POS_OUT_012 ::FOR POS_OUT_XBL_df Test case pass
 Out[ 23 ]: DataFrame[]

In [21]:
# ************************** DS_POS_OUT_013
#  DS_POS_OUT_013 :: Scenario ::  The Column 'PROGRAM_GROUP_NAME' should be 'WARRANTY', 'EMISSIONS' or 'WARRANTY_EMISSIONS' or All as Values
def DS_POS_OUT_013(POS_OUT_XBL_df):
  Pro_name = POS_OUT_XBL_df['PROGRAM_GROUP_NAME'].unique().tolist()
  c1=0  
  ls1=[]
  print(Pro_name)  
  given_ls = ['WARRANTY','EMISSIONS','WARRANTY_EMISSIONS','All']
  for j in Pro_name:    
    if j in given_ls : 
      c1+=1
    else:
      ls1.append(j)
  print(ls1)
  if len(ls1) == 0:  
      print("DS_POS_OUT_013 ::test case passed")
      log ='Test Case Passes'
  else:
      print("DS_POS_OUT_013 :: test case failed")
      log ='The Column "PROGRAM_GROUP_NAME" is having other values also ::Test Case Fail'
      TestFlag[13]=False
      CriticalTestFlag[13]=TestFlag[13] and CriticalTable['DS_POS_OUT_013']=="Critical"
  return TestFlag[13],log

my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_013(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_013'
nature_of_tc=CriticalTable['DS_POS_OUT_013']
metrics= "PROGRAM_GROUP_NAME"
value='The Column should have WARRANTY, EMISSIONS, WARRANTY_EMISSIONS or All as values'

if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log = str(sree_o[1])

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")
  

['All', 'WARRANTY', 'EMISSIONS', 'WARRANTY_EMISSIONS']
[]
DS_POS_OUT_013 ::test case passed
 Out[ 24 ]: DataFrame[]

In [22]:
# ************************** DS_POS_OUT_014
#  DS_POS_OUT_014 :: Scenario ::  The 'OM_METRIC' column should have 'COST_NBR', 'COUNT_NBR', 'CPE', 'RPH'.
def DS_POS_OUT_014(POS_OUT_XBL_df):
  Met_ls = POS_OUT_XBL_df['OM_METRIC'].unique().tolist()
  c1=0  
  ls1=[]
  print(Met_ls)  
  given_ls = ['COST_NBR', 'COUNT_NBR', 'CPE', 'RPH']
  for j in Met_ls:    
    if j in given_ls : 
      c1+=1
    else:
      ls1.append(j)
  print(ls1)
  if len(ls1) == 0:  
      print("DS_POS_OUT_014 ::test case passed")
      log ='Test Case Passes'
  else:
      print("DS_POS_OUT_014 :: test case failed")
      log ='The Column "OM_METRIC" is having other values ::Test Case Fail'
      TestFlag[14]=False
      CriticalTestFlag[14]=TestFlag[14] and CriticalTable['DS_POS_OUT_014']=="Critical"
  return TestFlag[14],log

my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_014(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_014'
nature_of_tc=CriticalTable['DS_POS_OUT_014']
metrics= "OM_METRIC"
value='The Column should have COST_NBR, COUNT_NBR, CPE, RPH as values'

if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log = str(sree_o[1])

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")
  

['COUNT_NBR', 'COST_NBR', 'RPH', 'CPE', 'WCPE']
['WCPE']
DS_POS_OUT_014 :: test case failed
 Out[ 25 ]: DataFrame[]

In [23]:
# *********************************** DS_POS_OUT_015
# Scenario :: The column 'OM_KEY' should be in the following format: CALC_ID-PROGRAM_GROUP_NAME-OM_METRIC-OM_ALGO

def DS_POS_OUT_015(POS_OUT_XBL_df):
    Om_Key = POS_OUT_XBL_df['OM_KEY'].tolist()
    New_Om_Key=POS_OUT_XBL_df['CALC_ID'].astype(str)+'-'+POS_OUT_XBL_df['PROGRAM_GROUP_NAME'].astype(str)+'-'+POS_OUT_XBL_df['OM_METRIC'].astype(str)+'-'+POS_OUT_XBL_df['OM_ALGO'].astype(str)
    
    if len(Om_Key) == len(New_Om_Key):
        c=0
        ls=[]
        for i,j in zip(Om_Key,New_Om_Key):
          if i == j:
            c+=1
          else:
            ls.append(i)
        if c == len(Om_Key) and len(ls)==0 :
            print("DS_POS_OUT_015 :: Test case pass")
            log ='Test Case Passes'
        else:
            print("DS_POS_OUT_015 :: Test case fail")
            log ='The Column "OM_KEY" not in specifed combination ::  Test Case Fail'
            TestFlag[15]=False
            CriticalTestFlag[15]=TestFlag[15] and CriticalTable['DS_POS_OUT_015']=="Critical"
        return TestFlag[15],log
    
my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_015(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_015'
nature_of_tc=CriticalTable['DS_POS_OUT_015']
metrics= "OM_KEY"
value='OM_KEY should be in specified format'
if sree_o[0] :
  status = 'Pass'
else:
  status = 'Fail'
failure_log = str(sree_o[1])

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")                


DS_POS_OUT_015 :: Test case fail
 Out[ 26 ]: DataFrame[]

In [24]:
# ************************** DS_POS_OUT_016
# Scenario :: The Column 'BUILD_YEAR' should be in 'YYYY' format or 'All'

def DS_POS_OUT_016(POS_OUT_XBL_df):
  from datetime import datetime
    
  date_lst = POS_OUT_XBL_df['BUILD_YEAR'].unique().tolist()  
  c= len(date_lst)
  #print(c)
  c1=0
  c2=0
  ls=[]
  c3=0
  for i in date_lst:
    if i !='All':   
      if datetime.strptime(i, '%Y'):
        c1+=1
      else:
        ls.append(i)
        c2+=1   

  if c1 == c or c1+1 ==c:
    print("DS_POS_OUT_016 ::  Test case pass")
    log="Test Case Passes"  
  else:
    print("DS_POS_OUT_016 :: Test case fail")
    log="Column values are not in format :: Test case Fail"
    #for i in ls:
    #    log_file(i)
    TestFlag[16]=False
    CriticalTestFlag[16]=TestFlag[16] and CriticalTable['DS_POS_OUT_016']=="Critical"
  return TestFlag[16],log

my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_016(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_016'
nature_of_tc=CriticalTable["DS_POS_OUT_016"]
metrics= "BUILD_YEAR"
value='The Column BUILD_YEAR should be in YYYY format or All'
if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log=str(sree_o[1])

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")


DS_POS_OUT_016 :: Test case pass
 Out[ 27 ]: DataFrame[]

In [25]:
# *********************************** DS_POS_OUT_018
#Scenario ::  The column REL_MONTH_BUILD_DATE should be in YYYY-MM-DD format.
def DS_POS_OUT_018(POS_OUT_XBL_df): 
  X_data = POS_OUT_XBL_df[POS_OUT_XBL_df['ENGINE_SERIES'] =='X']
  BnL_data = POS_OUT_XBL_df[POS_OUT_XBL_df['ENGINE_SERIES'] !='X']
  r_ls=[]
  c1=0
  def format_check(date):    
    if datetime.strptime(str(date), '%Y-%m') or datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S') :
      sree =1
    else:
      sree = 0
    return sree  
  if len(X_data) !=0:
    Xdf_cd = X_data['REL_MONTH_BUILD_DATE'].unique().tolist() 
    if len(Xdf_cd) ==1 and pd.isnull(Xdf_cd).all():
      print("For X Data :: Test Passed") 
      log = 'For X data ::only None Value is present :: Test Pass'      
      r_ls.append(log)
    else:
      log = 'For X data other than None values are present :: Test Fail'
      r_ls.append(log)
      TestFlag[18]=False
      CriticalTestFlag[18]=TestFlag[18] and CriticalTable['DS_POS_OUT_018']=="Critical" 
  
  if len(BnL_data)!=0:
    BnLdf_cd = BnL_data['REL_MONTH_BUILD_DATE'].unique().tolist() 
    for i in BnLdf_cd:
      if format_check(i) == 1:
        c1+=1
    if c1 == len(BnLdf_cd):
      print("For BnL Data :: Test Passed") 
      log = 'For BnL Data :: Test Case Passed'
      r_ls.append(log)
  else:
    print("For BnL Data :: Test case Failed") 
    log = 'For BnL Data :: Test Case Failed' 
    r_ls.append(log)  
    TestFlag[18]=False
    CriticalTestFlag[18]=TestFlag[18] and CriticalTable['DS_POS_OUT_018']=="Critical"      
  return TestFlag[18],r_ls

my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_018(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_018'
print(sree_o[1])
nature_of_tc=CriticalTable["DS_POS_OUT_018"]
metrics= "REL_MONTH_BUILD_DATE"
value='Column values should be in YYYY-MM-DD format for BnL Data and For X data column should have None Values'
if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log=str(sree_o[1])
df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")  


For BnL Data :: Test Passed
['For X data other than None values are present :: Test Fail', 'For BnL Data :: Test Case Passed']
 Out[ 31 ]: DataFrame[]

In [26]:
# *********************************** DS_POS_OUT_019
# Scenario :: The Column 'REL_QUARTER_BUILD_DATE' should be in 'YYYY-Q1'/'YYYY-Q2'/'YYYY-Q3'/'YYYY-Q4' formats or ALL
def DS_POS_OUT_019(POS_OUT_XBL_df):
  date_list = POS_OUT_XBL_df['REL_QUARTER_BUILD_DATE'].unique().tolist()
  def format_check(date_1):
      if datetime.strptime(date_1, '%Y'):
        Sree = 1
      return Sree
  ln=len(date_list)
  #print(ln)
  ls_1=[]
  ls_2=[]
  not_l=[]

  for i in date_list:
      if len(i) == 7:        
          lt=i.split('-')
          ls_1.append(lt[0])
          ls_2.append(lt[1])
      else:
          not_l.append(i)

  ls_Y=[]
  ls_Q=[]
  #print(ls_1)
  #print(ls_2)
  print(not_l)
  q_lst=['Q1','Q2','Q3','Q4']
  c=0 
  for i,j in zip(ls_1,ls_2):
      if format_check(i) == 1 and j in q_lst: 
          c+=1
      else:
          ls_Y.append(i)
          ls_Q.append(j)
  print(c)
  if c == ln or c+1==ln:
      print("DS_POS_OUT_019 :: Test case Pass")
      log="Test case Pass"  
  else:    
      print("DS_POS_OUT_019 ::  Test Fail")
      log="Column REL_QUARTER_BUILD_DATE values are not in format ::  Test Case Fail"
      #for i,j,k in zip(not_l,ls_Y,ls_Q):
       #   print(i)
       #   print(j)
       #  print(k) 
      TestFlag[19]=False
      CriticalTestFlag[19]=TestFlag[19] and CriticalTable['DS_POS_OUT_019']=="Critical"
  return TestFlag[19],log

my_date1 = datetime.now(pytz.timezone('US/Eastern'))
status= DS_POS_OUT_019(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_019'
nature_of_tc=CriticalTable["DS_POS_OUT_019"]
metrics= "REL_QUARTER_BUILD_DATE"
value='REL_QUARTER_BUILD_DATE should be in YYYY-Q1/YYYY-Q2/YYYY-Q3/YYYY-Q4 formats or All'
test_status=str(status[0])
failure_log=str(status[1])

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")

['All']
28
DS_POS_OUT_019 :: Test case Pass
 Out[ 32 ]: DataFrame[]

In [27]:
# *********************************** DS_POS_OUT_020
#Scenario ::  The duplicate row data should not be present in preprocessing files
def DS_POS_OUT_020(POS_OUT_XBL_df):
    
    Group_df = POS_OUT_XBL_df.groupby(["CALC_ID","CALC_DATE"]).size().reset_index(name='count_1')
    unique_count = Group_df['count_1'].unique().tolist()
    if len(unique_count) == 1 and 1 in unique_count:
        print('DS_POS_OUT_020 ::No Duplicate records present ::Test case Pass')
        log='Test case Pass'
    else:
        print("DS_POS_OUT_020 :: Duplicate records present :: Test case Failed")
        log="The Table is having duplicate Records for Primary Key CALC_ID + CALC_DATE ::  Test Case Fail"
        TestFlag[20]=False
        CriticalTestFlag[20]=TestFlag[20] and CriticalTable['DS_POS_OUT_020']=="Critical"
    return TestFlag[20],log 

my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_020(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_020'
nature_of_tc=CriticalTable["DS_POS_OUT_020"]
metrics= "CALC_ID & CALC_DATE"
value='Data should not have Duplicate records for specified primary key'
if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log=str(sree_o[1])
df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")  

DS_POS_OUT_020 :: Duplicate records present :: Test case Failed
 Out[ 33 ]: DataFrame[]

In [28]:
# ************************** DS_POS_OUT_021
#Scenario :: CALC_DATE , EARLIEST_OCCURRENCE_DATE, OM_RUN_ANALYTICS_DATE and OM_OUTPUT_1 should be in YYYY-MM-DD format.

def DS_POS_OUT_021(POS_OUT_XBL_df):  
  def format_check(date):
    if str(date) !='NaT' or str(date)!= 'None' or datetime.strptime(str(date), '%Y-%m-%d') or datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S'):
      sree =1
    else:
      sree = 0
    return sree
  df_cd = POS_OUT_XBL_df['CALC_DATE'].astype(str).unique().tolist()
  df_eod = POS_OUT_XBL_df['EARLIEST_OCCURRENCE_DATE'].astype(str).unique().tolist()
  df_ornd = POS_OUT_XBL_df['OM_RUN_ANALYTICS_DATE'].astype(str).unique().tolist()
  df_oout = POS_OUT_XBL_df['OM_OUTPUT_01'].astype(str).unique().tolist()
  c1=0
  c2=0
  c3=0 
  c4 =0
  for i in df_cd:
    if format_check(i) == 1:
      c1+=1
  for j in df_eod:
    if format_check(j) ==1:
      c2+=1      
  for k in df_ornd:
    if format_check(j) ==1:
      c3+=1
  for s in df_oout:
    if format_check(s) ==1:
      c4+=1

  #print(c1,c2,c3)
  #print(len(df_cd),len(df_eod))
  r_ls=[]
  if c1 == len(df_cd):
    print("DS_POS_OUT_021 :: For CALC_DATE :: Test case Passed")  
    log = 'For CALC_DATE :: Test Case Passes'
    r_ls.append(log)
  else:
    print("DS_POS_OUT_021 :: For CALC_DATE :: Test case Failed")  
    log = 'CALC_DATE Column Values are not in Format :: Test Case Fail '
    r_ls.append(log)
    TestFlag[21]=False
    CriticalTestFlag[21]=TestFlag[21] and CriticalTable['DS_POS_OUT_021']=="Critical"      
  if c2 == len(df_eod):
    print("DS_POS_OUT_021 :: For EARLIEST_OCCURRENCE_DATE :: Test case Passed") 
    log = 'For EARLIEST_OCCURRENCE_DATE :: Test Case Passes'
    r_ls.append(log)
  else:
    print("DS_POS_OUT_021 :: For EARLIEST_OCCURRENCE_DATE :: Test case Failed")
    log = 'EARLIEST_OCCURRENCE_DATE Column Values are not in Format :: Test Case Fail '
    r_ls.append(log)
    TestFlag[21]=False
    CriticalTestFlag[21]=TestFlag[21] and CriticalTable['DS_POS_OUT_021']=="Critical" 
  if c3 == len(df_ornd):
    print("DS_POS_OUT_021 :: For OM_RUN_ANALYTICS_DATE :: Test case Passed") 
    log = 'For OM_RUN_ANALYTICS_DATE :: Test Case Passes' 
    r_ls.append(log)
  else:
    print("DS_POS_OUT_021 :: For OM_RUN_ANALYTICS_DATE :: Test case Failed")
    log = 'OM_RUN_ANALYTICS_DATE Column Values are not in Format :: Test Case Fail '
    r_ls.append(log)
    TestFlag[21]=False
    CriticalTestFlag[21]=TestFlag[21] and CriticalTable['DS_POS_OUT_021']=="Critical"
  if c4 == len(df_oout):
    print("DS_POS_OUT_021 :: For OM_OUTPUT_1  :: Test case Passed") 
    log = 'For OM_OUTPUT_1 :: Test Case Passes' 
    r_ls.append(log)
  else:
    print("DS_POS_OUT_021 :: For OM_OUTPUT_1 :: Test case Failed")
    log = 'OM_OUTPUT_1 Column Values are not in Format :: Test Case Fail '
    r_ls.append(log)
    TestFlag[21]=False
    CriticalTestFlag[21]=TestFlag[21] and CriticalTable['DS_POS_OUT_021']=="Critical"
  return TestFlag[21],r_ls

my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_021(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_021'
nature_of_tc=CriticalTable["DS_POS_OUT_021"]
metrics= "CALC_DATE, EARLIEST_OCCURRENCE_DATE & RUN_ANALYTICS_DATE"
value='Should be in YYYY-MM-DD format.'
if sree_o[0] :
  test_status = 'Pass'
else:
  test_status = 'Fail'
failure_log=str(sree_o[1])

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,test_status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")


DS_POS_OUT_021 :: For CALC_DATE :: Test case Passed
DS_POS_OUT_021 :: For EARLIEST_OCCURRENCE_DATE :: Test case Passed
DS_POS_OUT_021 :: For OM_RUN_ANALYTICS_DATE :: Test case Passed
DS_POS_OUT_021 :: For OM_OUTPUT_1 :: Test case Passed
 Out[ 34 ]: DataFrame[]

In [29]:
# *********************** DS_POS_OUT_022
#Scenario :: No rows/columns data should have NULL values
def DS_POS_OUT_022(POS_OUT_XBL_df):
  def null_check(col_name):
    if pd.isnull(POS_OUT_XBL_df[col_name]).any():
      sree =1 
    else:
      sree =0
    return sree
  col_names = POS_OUT_XBL_df.columns.tolist()
  col_names = [str.upper(cval) for cval in col_names]
  c=0
  c1=0
  ls1=[]
  ls2=[]
  for i in col_names:
    if null_check(i) == 1:
      c+=1
      ls1.append(i)
    else:
      c1+=1
      ls2.append(i)

  print(c,c1,len(col_names))  
  r_ls = []
  if c == len(col_names) and c1 == 0:
    print('All columns are not null')
    log="For All Columns Test case passed" 
    r_ls.append(log)
    for i in ls1 :     
      print(" DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == " + i + " == Test case passed")      
  else:
    print('Some columns are null and some columns are not null')    
    TestFlag[22]=False
    CriticalTestFlag[22]=TestFlag[22] and CriticalTable['DS_POS_OUT_022']=="Critical"
    for i in ls1 :
      print(" DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == " + i + " == Test case passed")
      log="For Column == " + i + " == Test case passed"
      r_ls.append(log)
    for j in ls2 :
      print("DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == " + j + " == Test case Failed")
      log="For Column == " + j + " == Test case Failed"
      r_ls.append(log)
      #print(len(ls2),len(ls1)) 
  
  return TestFlag[22],r_ls
my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o=  DS_POS_OUT_022(POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_022'
nature_of_tc=CriticalTable['DS_POS_OUT_022']
metrics= "All Columns "
value='No rows/columns data should have NULL values'
if sree_o[0] :
  status = 'Pass'
else:
  status = 'Fail'
failure_log = str(sree_o[1])

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df") 

  


13 27 40
Some columns are null and some columns are not null
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_OUTPUT_01 == Test case passed
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_OUTPUT_02 == Test case passed
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_OUTPUT_03 == Test case passed
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_NBR_PREV == Test case passed
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_SCORE == Test case passed
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_SCORE_DELTA == Test case passed
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_WEIBULL_SCORE_RPH_MILEAGE_MEDIAN == Test case passed
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_WEIBULL_SCORE_RPH_MONTH_IN_SERVICE_MEDIAN == Test case passed
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == REL_USER_APPL_DESC == Test case passed
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == EARLIEST_OCCURRENCE_DATE == Test case passed
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == REL_MONTH_BUILD_DATE == Test case passed
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == WEIBULL_RPH_MONTH_IN_SERVICE_MEDIAN == Test case passed
 DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == WEIBULL_RPH_MILEAGE_MEDIAN == Test case passed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == CALC_ID == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == CALC_DATE == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == CODE == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_CRRT_PRIORITY_SCORE == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_PREV_PRIORITY_SCORE == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == SOURCE == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_ALGO == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_RUN_ANALYTICS_DATE == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_NBR == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_SCORE_REL == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_SCORE_REL_PREV == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_SCORE_REL_BUCKET == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_SCORE_REL_PREV_BUCKET == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == ENGINE_SERIES == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == REL_ENGINE_NAME_DESC == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == PROGRAM_GROUP_NAME == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_DELTA == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_METRIC == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_KEY == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_JIRA_URL == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == BUILD_VOLUME == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == BUILD_YEAR == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_NBR_CUMUL == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_NBR_CUMUL_BUCKET == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == OM_CRRT_PRIORITY_SCORE_BUCKET == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == REL_OEM_NORMALIZED_GROUP == Test case Failed
DS_POS_OUT_022 :: For POS_OUT_XBL_df Column name == REL_QUARTER_BUILD_DATE == Test case Failed
 Out[ 35 ]: DataFrame[]

In [30]:
col_ls = Algo_XBL.dtypes
col_names = Algo_XBL_df.columns.tolist()
ls=["string", "date", "string", "double", "double", "string", "string", "string", "date", "string", "string", "double", "double", "string", "double", "double", "double", "string", "string", "string", "string", "string", "string", "string", "integer", "string", "string", "string", "string", "string", "string", "date", "string", "double", "double", "string"]
c=0
ls_n=[]
for i in range(0,len(col_ls)):
  if col_ls[i][1] == ls[i]:
    c+=1
  else:
    ls_n.append(col_ls[i][0])
print(ls_n,c)
  
    

['SCORE', 'BUILD_VOLUME', 'REL_MONTH_BUILD_DATE', 'RUN_ANALYTICS_DATE', 'METRIC_VALUE_CUMUL_PREV', 'DELTA'] 28

In [31]:
# ************************** DS_POS_OUT_023
#Scenario ::Data Type Check for each column

def DS_POS_OUT_023(POS_OUT_XBL,POS_OUT_XBL_df):  
  col_ls = POS_OUT_XBL.dtypes
  col_names = POS_OUT_XBL_df.columns.tolist()
  #print(col_ls)
  #ls=["string", "date", "string", "float", "float", "date", "float", "float","string", "string", "date", "float", "float", "float", "float", "float", "float", "string", "string", "float", "float","string","string", "string", "float", "string", "string", "string", "int", "string", "string", "date", "float", "string", "string", "string", "string", "float", "float", "string"]
  ls=["string", "timestamp", "string", "double", "double", "timestamp", "double", "double","string", "string", "timestamp", "double", "double", "double", "double", "double", "double", "string", "string", "double", "double","string","string", "string", "double", "string", "string", "string", "int", "string", "string", "timestamp", "double", "string", "string", "string", "string", "double", "double", "string"]
  ls_main = []
  c=0
  ls_n=[]
  for i in range(0,len(col_ls)):
    if col_ls[i][1] == ls[i]:
      c+=1
    else:
      ls_n.append(col_ls[i][0])  

  #print(ls_n)
  r_ls=[]
  if c == len(col_ls) and len(ls_n)==0:
    print("DS_POS_OUT_023 :: For POS_OUT_XBL_df test case passed")
    log = 'For All Columns Test Case Passes'
    r_ls.append(log)    
  else:
    print("DS_POS_OUT_023 :: For POS_OUT_XBL_df test case failed")    
    TestFlag[23]=False
    CriticalTestFlag[23]=TestFlag[23] and CriticalTable['DS_POS_OUT_023']=="Critical" 
    for i in ls_n:
      log = 'Column ='+i+ ' Test is failed'
      print(log)
      r_ls.append(log)      
    
  return TestFlag[23],r_ls
    
my_date1 = datetime.now(pytz.timezone('US/Eastern'))
sree_o= DS_POS_OUT_023(POS_OUT_XBL,POS_OUT_XBL_df)
my_date2 = datetime.now(pytz.timezone('US/Eastern'))
td=(my_date2-my_date1).total_seconds()
testcase_id='DS_POS_OUT_023'
nature_of_tc=CriticalTable['DS_POS_OUT_023']
metrics= "All Columns"
value='Data Type Check for each column'
if sree_o[0] :
  status = 'Pass'
else:
  status = 'Fail'
failure_log = str(sree_o[1])

df=sql("select * from quality_qa_mi.preprocessing_validate")
insert_df = spark.createDataFrame([(myrunId,testcase_id,my_date1,my_date2,td,nature_of_tc,metrics,value,status,failure_log)], df.schema)
insert_df.registerTempTable("insert_df") 
sql("insert into table quality_qa_mi.preprocessing_validate select * from insert_df")
  

DS_POS_OUT_023 :: For POS_OUT_XBL_df test case passed
 Out[ 36 ]: DataFrame[]

In [32]:
%sql
select * from quality_qa_mi.preprocessing_validate Order by start_time desc

runid,testcase_id,start_time,end_time,execution_time,nature_of_tc,metrics,description,status,failure_log
1563283411,DS_POS_OUT_023,2019-07-16T13:57:16.670+0000,2019-07-16T13:57:16.670+0000,2.71E-4,Not_Critical,All Columns,Data Type Check for each column,Pass,['For All Columns Test Case Passes']
1563283411,DS_POS_OUT_022,2019-07-16T13:56:47.821+0000,2019-07-16T13:56:53.182+0000,5.360302,Not_Critical,All Columns,No rows/columns data should have NULL values,Fail,"['For Column == OM_OUTPUT_01 == Test case passed', 'For Column == OM_OUTPUT_02 == Test case passed',"
1563283411,DS_POS_OUT_021,2019-07-16T13:56:12.819+0000,2019-07-16T13:56:43.824+0000,31.005175,Not_Critical,"CALC_DATE, EARLIEST_OCCURRENCE_DATE & RUN_ANALYTIC",Should be in YYYY-MM-DD format.,Pass,"['For CALC_DATE :: Test Case Passes', 'For EARLIEST_OCCURRENCE_DATE :: Test Case Passes', 'For OM_RU"
1563283411,DS_POS_OUT_020,2019-07-16T13:55:59.319+0000,2019-07-16T13:56:00.629+0000,1.309444,Not_Critical,CALC_ID & CALC_DATE,Data should not have Duplicate records for specifi,Fail,The Table is having duplicate Records for Primary Key CALC_ID + CALC_DATE :: Test Case Fail
1563283411,DS_POS_OUT_019,2019-07-16T13:52:30.328+0000,2019-07-16T13:52:30.695+0000,0.366707,Not_Critical,REL_QUARTER_BUILD_DATE,REL_QUARTER_BUILD_DATE should be in YYYY-Q1/YYYY-Q,True,Test case Pass
1563283411,DS_POS_OUT_018,2019-07-16T13:52:13.986+0000,2019-07-16T13:52:16.846+0000,2.86002,Not_Critical,REL_MONTH_BUILD_DATE,Column values should be in YYYY-MM-DD format for B,Fail,"['For X data other than None values are present :: Test Fail', 'For BnL Data :: Test Case Passed']"
1563283411,DS_POS_OUT_016,2019-07-16T13:49:40.863+0000,2019-07-16T13:49:41.237+0000,0.374313,Not_Critical,BUILD_YEAR,The Column BUILD_YEAR should be in YYYY format or,Pass,Test Case Passes
1563283411,DS_POS_OUT_015,2019-07-16T13:48:47.074+0000,2019-07-16T13:48:51.253+0000,4.178392,Not_Critical,OM_KEY,OM_KEY should be in specified format,Fail,"The Column ""OM_KEY"" not in specifed combination :: Test Case Fail"
1563283411,DS_POS_OUT_014,2019-07-16T13:48:34.095+0000,2019-07-16T13:48:34.420+0000,0.324941,Not_Critical,OM_METRIC,"The Column should have COST_NBR, COUNT_NBR, CPE, R",Fail,"The Column ""OM_METRIC"" is having other values ::Test Case Fail"
1563283411,DS_POS_OUT_013,2019-07-16T13:48:27.347+0000,2019-07-16T13:48:27.654+0000,0.306717,Not_Critical,PROGRAM_GROUP_NAME,"The Column should have WARRANTY, EMISSIONS, WARRAN",Pass,Test Case Passes
